In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score

In [ ]:
df=pd.read_csv('../input/spaceship-titanic/train.csv')
test_set=pd.read_csv('../input/spaceship-titanic/test.csv')

In [ ]:
df.head()

In [ ]:
test_set.head()

In [ ]:
df.info()

Checking for all Categorical Values

In [ ]:
cat=[]
for col in df.columns:
    print(col)
    ds=df[col].unique().shape
    print(ds[0])
    cat.append((col,ds[0]))
    print('-------')

In [ ]:
print(cat)

In [ ]:
category=[]
for i in cat:
    if i[0]=='Transported':
        continue
    if i[1]<6:
        category.append(i[0])
category        

In [ ]:
df[category] = df[category].astype('category')
df['Transported']= df['Transported'].astype('object')

In [ ]:
df.info()

In [ ]:
to_drop=['PassengerId','Cabin','Name']
df=df.drop(to_drop,axis=1)

In [ ]:
df.head()

In [ ]:
df.isna().sum()

Substituing Mean values for every Continuous Feature

In [ ]:
df['Age']=df['Age'].fillna(df['Age'].mean())
df['RoomService']=df['RoomService'].fillna(df['RoomService'].mean())
df['FoodCourt']=df['FoodCourt'].fillna(df['FoodCourt'].mean())
df['ShoppingMall']=df['ShoppingMall'].fillna(df['ShoppingMall'].mean())
df['Spa']=df['Spa'].fillna(df['Spa'].mean())
df['VRDeck']=df['VRDeck'].fillna(df['VRDeck'].mean())

In [ ]:
df.isna().sum()

In [ ]:
df=df.dropna()

In [ ]:
df

In [ ]:
df['Transported']=pd.get_dummies(df['Transported'],drop_first=True)

In [ ]:
df

In [ ]:
y=df['Transported']
df=df.drop('Transported',axis=1)

In [ ]:
categorical_columns=['HomePlanet','CryoSleep','Destination','VIP']
cat_col=df[categorical_columns]
df=df.drop(categorical_columns,axis=1)

Encoding using Get Dummies

In [ ]:
cat_col=pd.get_dummies(cat_col,drop_first=True)

In [ ]:
cat_col

In [ ]:
df_new = pd.concat([df, cat_col], axis=1)

In [ ]:
df_new

In [ ]:
from sklearn.preprocessing import scale
X_scaled=scale(df_new)

In [ ]:
X_scaled

In [ ]:
y

In [ ]:
test_set

In [ ]:
test_set.isna().sum()

Removing PassengerID, Cabin, Name as these features don't help the model 

In [ ]:
to_drop=['PassengerId','Cabin','Name']
test=test_set.drop(to_drop,axis=1)

In [ ]:
test

In [ ]:
test['Age']=test['Age'].fillna(test['Age'].mean())
test['RoomService']=test['RoomService'].fillna(test['RoomService'].mean())
test['FoodCourt']=test['FoodCourt'].fillna(test['FoodCourt'].mean())
test['ShoppingMall']=test['ShoppingMall'].fillna(test['ShoppingMall'].mean())
test['Spa']=test['Spa'].fillna(test['Spa'].mean())
test['VRDeck']=test['VRDeck'].fillna(test['VRDeck'].mean())

In [ ]:
test.isna().sum()

In [ ]:
test

In [ ]:
cat_col=test[categorical_columns]
test=test.drop(categorical_columns,axis=1)

In [ ]:
cat_col=pd.get_dummies(cat_col,drop_first=True)

In [ ]:
cat_col

In [ ]:
test_new = pd.concat([test, cat_col], axis=1)

In [ ]:
test_new

In [ ]:
test=scale(test_new)

In [ ]:
test

# Logistic Regression and CART

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
lr=LogisticRegression()
dt=DecisionTreeClassifier(max_depth=10)
classifiers=[('Logistic Regression',lr),('CART',dt)]

In [ ]:
for clf_name,clf in classifiers:
    clf.fit(X_scaled,y)
    y_pred=clf.predict(X_scaled)
    print(f'Train Accuracy:{clf.score(X_scaled,y)}')


In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
for clf_name,clf in classifiers:
    y_pred_proba=clf.predict_proba(X_scaled)[:,1]
    fpr,tpr,thresholds=roc_curve(y,y_pred_proba)
    plt.plot(fpr,tpr)
    plt.xlabel('FPR')
    plt.ylabel('TPR')
    plt.title(clf_name)
    plt.show()
    print(f'AUC SCORE:{roc_auc_score(y,y_pred_proba)}')
#     print(f'F1 Score: {f1_score(y, y_pred)}')

# Voting Classifier

In [ ]:
vc=VotingClassifier(estimators=classifiers,voting='soft')
vc.fit(X_scaled,y)
y_pred=vc.predict(X_scaled)
print(f'Train Accuracy:{vc.score(X_scaled,y)}')

In [ ]:
from sklearn.metrics import roc_curve
y_pred_proba=vc.predict_proba(X_scaled)[:,1]
fpr,tpr,thresholds=roc_curve(y,y_pred_proba)
plt.plot(fpr,tpr)
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y,y_pred_proba)

In [ ]:
f1_score(y, y_pred)

# Bagging

In [ ]:
from sklearn.ensemble import BaggingClassifier
dt=DecisionTreeClassifier()
bc=BaggingClassifier(base_estimator=dt,n_estimators=300,n_jobs=-1)
bc.fit(X_scaled,y)
y_pred=bc.predict(X_scaled)
print(bc.score(X_scaled,y))

In [ ]:
from sklearn.metrics import roc_curve
y_pred_proba=bc.predict_proba(X_scaled)[:,1]
fpr,tpr,thresholds=roc_curve(y,y_pred_proba)
plt.plot(fpr,tpr)
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y,y_pred_proba)

In [ ]:
f1_score(y, y_pred)

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(n_estimators=80,n_jobs=-1)
rf.fit(X_scaled,y)
y_pred=rf.predict(X_scaled)
print(rf.score(X_scaled,y))

In [ ]:
from sklearn.metrics import roc_curve
y_pred_proba=rf.predict_proba(X_scaled)[:,1]
fpr,tpr,thresholds=roc_curve(y,y_pred_proba)
plt.plot(fpr,tpr)
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y,y_pred_proba)

In [ ]:
f1_score(y, y_pred)

# AdaBoostClassifier

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
ad=AdaBoostClassifier(base_estimator=dt)
ad.fit(X_scaled,y)
y_pred=ad.predict(X_scaled)
print(ad.score(X_scaled,y))

In [ ]:
from sklearn.metrics import roc_curve
y_pred_proba=ad.predict_proba(X_scaled)[:,1]
fpr,tpr,thresholds=roc_curve(y,y_pred_proba)
plt.plot(fpr,tpr)
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y,y_pred_proba)


In [ ]:
f1_score(y, y_pred)

# GradientBoosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier(n_estimators=300,max_depth=100)
gbc.fit(X_scaled,y)
y_pred=gbc.predict(X_scaled)
print(gbc.score(X_scaled,y))

In [ ]:
from sklearn.metrics import roc_curve
y_pred_proba=gbc.predict_proba(X_scaled)[:,1]
fpr,tpr,thresholds=roc_curve(y,y_pred_proba)
plt.plot(fpr,tpr)
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y,y_pred_proba)


In [ ]:
f1_score(y, y_pred)

Using Gradient Boosting to predict the transported passengers

In [ ]:
final_pred=gbc.predict(test)

In [ ]:
a=test_set[['PassengerId']]
a

In [ ]:
a['Transported']=final_pred

In [ ]:
change={0:'False',1:'True'}
a['Transported']=a['Transported'].replace(change)

In [ ]:
a.to_csv('submission.csv', index=False)